In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=postgres password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

#### Process song data

In [4]:
song_files = get_files('./data/song_data/')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


#### Song table

In [36]:
song_data_df = df[['song_id', 'title', 'year', 'artist_id', 'duration']]
song_data_values = song_data_df.values
first_record_df = song_data_values[0]
song_data = first_record_df.tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 1982,
 'AR7G5I41187FB4CE6C',
 233.40363]

In [26]:
# %load create_tables.py
import psycopg2
from sql_queries import *


def create_database():
    # connect to default database
    conn = psycopg2.connect(
        "host=127.0.0.1 dbname=studentdb user=postgres password=student")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS sparkdb1")
    cur.execute(
        "CREATE DATABASE sparkdb1 WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()

    # connect to sparkify database
    conn = psycopg2.connect(
        "host=127.0.0.1 dbname=sparkdb1 user=postgres password=student")
    cur = conn.cursor()
    return cur, conn
    


def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    cur, conn = create_database()

    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()


In [34]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkdb1 user=postgres password=student")
cur = conn.cursor()

In [30]:
from sql_queries import *

In [35]:
cur.execute(song_table_insert, song_data)
conn.commit()

IndexError: list index out of range